# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
from functools import partial
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

load_dotenv(override=True)

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    database="sakila"
)

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print("Executando query:")
        cursor.execute(query, args)
        for result in cursor:
            print(result)

db = partial(run_db_query, connection)

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [ ]:
ia.tasks()

In [ ]:
ia.grades(by="task")

In [ ]:
ia.grades(task="views")

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Mostre o nome da categoria e a receita. Ordene da maior receita para para a menor.

In [ ]:
sql_ex01 = """
SELECT category.name, SUM(payment.amount) as total_sales
FROM category
JOIN film_category ON category.category_id = film_category.category_id
JOIN film ON film_category.film_id = film.film_id
JOIN inventory ON film.film_id = inventory.film_id
JOIN rental ON inventory.inventory_id = rental.inventory_id
JOIN payment ON rental.rental_id = payment.rental_id
GROUP BY category.name
ORDER BY total_sales DESC;
"""

db(sql_ex01)

In [ ]:
ia.sender(answer="sql_ex01", task="views", question="ex01", answer_type="pyvar")

Liste os 10 filmes mais alugados e o numero de vezes em que foram alugados. Ordene do mais alugado para o menos alugado.

In [ ]:
sql_ex02 = """
SELECT film.title, COUNT(rental.rental_id) as total_rentals
FROM film
JOIN inventory ON film.film_id = inventory.film_id
JOIN rental ON inventory.inventory_id = rental.inventory_id
GROUP BY film.title
ORDER BY total_rentals DESC
LIMIT 10;
"""

db(sql_ex02)

In [ ]:
ia.sender(answer="sql_ex02", task="views", question="ex02", answer_type="pyvar")

**DESAFIO:** Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme. *Dica*: Serão necessários múltiplos SELECTs. Faça cada um em uma célula diferente.

In [ ]:
# db("""
# SELECT AVG(payment.amount) INTO @avg_payment
# FROM category
# JOIN film_category ON category.category_id = film_category.category_id
# JOIN film ON film_category.film_id = film.film_id
# JOIN inventory ON film.film_id = inventory.film_id
# JOIN rental ON inventory.inventory_id = rental.inventory_id
# JOIN payment ON rental.rental_id = payment.rental_id
# GROUP BY category.name
# ORDER BY total_sales DESC;
# """)
# db("""
# SELECT category.name, SUM(payment.amount) as total_sales
# FROM category
# JOIN film_category ON category.category_id = film_category.category_id
# JOIN film ON film_category.film_id = film.film_id
# JOIN inventory ON film.film_id = inventory.film_id
# JOIN rental ON inventory.inventory_id = rental.inventory_id
# JOIN payment ON rental.rental_id = payment.rental_id
# GROUP BY category.name
# HAVING total_sales > @avg_payment
# ORDER BY total_sales DESC;
# """)
db("""
-- SUA QUERY AQUI!
""")
db("""
-- SUA QUERY AQUI!
""")

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [ ]:
db("""
DROP VIEW IF EXISTS movie_count;
""")

db("""
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
""")

Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [ ]:
db("""
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
""")

In [ ]:
db("""
SELECT 
    * 
FROM 
    movie_count 
ORDER BY
    cnt DESC
LIMIT 30
""")

Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [ ]:
db("""  
UPDATE film
SET title = 'stoned punk'
WHERE title = 'dazed punk'
""")

Verifique agora a nossa *view*:

In [ ]:
db("""
SELECT
    *
FROM
    movie_count
LIMIT
    30
""")

Como você pode ver, as views são tabelas virtuais que são **automaticamente atualizadas quando as tabelas originais são modificadas**.

Sempre que você realizar modificações nos dados, dê `commit` ou `rollback`. Ainda, evite executar múltiplas vezes as linhas de código que criem a conexão sem antes ter fechado a conexão ativa.

Vamos desfazer as alterações:

In [ ]:
connection.rollback()

Conferindo:

In [ ]:
db("""
SELECT
    *
FROM
    movie_count
LIMIT
    30
""")

### Vamos praticar

Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [ ]:
sql_ex03 = '''
-- how many times "COWBOY DOOM" was rented using the view movie_count
SELECT 
    sum(cnt) as total_rentals
FROM
    movie_count
WHERE
    title = 'COWBOY DOOM'


'''

db(sql_ex03)

In [ ]:
ia.sender(answer="sql_ex03", task="views", question="ex03", answer_type="pyvar")

Registre um aluguel do filme "COWBOY DOOM" (com o menor id possível), feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'.

In [ ]:
sql_ex04 = ('''
SELECT film_id
FROM film
WHERE title = 'COWBOY DOOM';
''')
db(sql_ex04)

sql_ex04 = ('''
SELECT '2019-01-01', film.film_id, customer.customer_id, '2019-01-08', staff.staff_id
FROM film, customer, staff
WHERE film.title = 'COWBOY DOOM' 
    AND customer.first_name = 'Jessie' AND customer.last_name = 'Banks'
    AND staff.first_name = 'Jon' AND staff.last_name = 'Stephens';
''')
db(sql_ex04)

In [ ]:
sql_ex04 = ('''
INSERT INTO rental (rental_date, inventory_id, customer_id, return_date, staff_id)
SELECT '2019-01-01', 845, customer.customer_id, '2019-01-08', staff.staff_id
FROM film, customer, staff
WHERE film.title = 'COWBOY DOOM' 
    AND customer.first_name = 'Jessie' AND customer.last_name = 'Banks'
    AND staff.first_name = 'Jon' AND staff.last_name = 'Stephens'
''')
db(sql_ex04)

In [ ]:
ia.sender(answer="sql_ex04", task="views", question="ex04", answer_type="pyvar")

Verifique usando a view que a contagem de alugueis do filme subiu.

In [ ]:
db("""
-- SUA QUERY AQUI!
""")

Faça o *rollback* desta alteração, para não modificar a nossa querida base de dados *sakila*.

In [ ]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [ ]:
db("""
DROP TABLE IF EXISTS long_film
""")

db("""
CREATE TEMPORARY TABLE long_film 
    SELECT
        *
    FROM
        film
    WHERE
        film.length > 180;
""")

Podemos verificar que a tabela `long_film` agora existe:

In [ ]:
db("DESCRIBE long_film")

In [ ]:
db("CALL sys.table_exists('sakila', 'long_film', @table_type);")
db("SELECT @table_type;")

Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [ ]:
db("SHOW TABLES")

Vamos listar o conteudo desta tabela:

In [ ]:
db("SELECT title FROM long_film")

Vamos apagar a tabela `long_film`:

In [ ]:
db("DROP TABLE long_film")

### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria. Apresente o id da categoria, seu nome e a duração máxima.

In [ ]:
sql_ex = """
DESCRIBE category;
"""

db(sql_ex)

In [ ]:
# Executamos o DROP apenas localmente, sem enviar ao servidor
db("DROP TABLE IF EXISTS max_duration")

sql_ex05 = """
CREATE TEMPORARY TABLE max_duration
    SELECT category.category_id, category.name, max(film.length)
    FROM category
        JOIN film_category USING (category_id)
        JOIN film USING (film_id)
    GROUP BY category.name, category.category_id;
"""

db(sql_ex05)

In [ ]:
sql_ex = """
SELECT * FROM max_duration;
"""

db(sql_ex)

In [ ]:
ia.sender(answer="sql_ex05", task="views", question="ex05", answer_type="pyvar")

 - Verifique a tabela.

In [ ]:
db("SELECT * FROM max_duration")

In [ ]:
db("DESCRIBE max_duration")

- Agora use a tabela temporária para construir uma consulta com as categorias e seus respectivos filmes mais longos:

In [ ]:
# db("""
# SELECT
#     category_id, name, film_id, title, length
# FROM
#     film
#     INNER JOIN film_category USING (film_id)
#     INNER JOIN max_duration USING (category_id)
# WHERE
#     length = max_len
# """)

- delete a tabela temporária

In [ ]:
db("""
DROP TABLE max_duration
""")

## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [ ]:
db("""
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
""")

Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [ ]:
db("SELECT @avg_rate, @stddev_rate")

Agora podemos selecionar os filmes caros!

In [ ]:
db("""
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate
LIMIT 10
""")

### Vamos praticar

Armazene na variável temporária `max_films` a quantidade de filmes feitos pelo ator ou atriz que mais participou de filmes.

In [ ]:
sql_ex06 = """
SELECT @max_films := MAX(cnt)
FROM (SELECT actor_id, COUNT(film_id) as cnt
    FROM film_actor
    GROUP BY actor_id) 
AS actor_movies;
"""

db(sql_ex06)

In [ ]:
db("""
SELECT @max_films
""")

In [ ]:
ia.sender(answer="sql_ex06", task="views", question="ex06", answer_type="pyvar")

## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [ ]:
db("""
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
""")

Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [ ]:
db("DROP TABLE IF EXISTS temp_pop_actors")
db("""
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT first_name, last_name, actor_id FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    GROUP BY 
        actor_id
    ORDER BY 
        COUNT(film_id) DESC
    LIMIT 3
""")
db("""
SELECT * from temp_pop_actors
""")

Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [ ]:
db("""
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
""")

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [ ]:
db("DROP TABLE temp_pop_actors")

### Vamos praticar

Liste os atores (id, nome e sobrenome) que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora) ordenados pelo id do ator de modo crescente. Para isso, crie uma tabela temporária contendo o id do filme e a quantia total e use essa tabela para listar os atores.

In [ ]:
db("""
-- SUA QUERY AQUI!
""")
db("""
-- SUA QUERY AQUI!
""")

In [ ]:
sql_ex07 = ["""
CREATE TEMPORARY TABLE top_movies_revenue AS
    SELECT f.film_id, f.title AS film_title, SUM(p.amount) AS total_revenue
    FROM film f
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    JOIN payment p ON r.rental_id = p.rental_id
    GROUP BY f.film_id
    ORDER BY total_revenue DESC
    LIMIT 3;
""",
"""
SELECT DISTINCT a.actor_id, a.first_name, a.last_name
    FROM actor a
    JOIN film_actor fa ON a.actor_id = fa.actor_id
    JOIN film f ON fa.film_id = f.film_id
    JOIN top_movies_revenue tmr ON f.film_id = tmr.film_id
    ORDER BY a.actor_id, a.first_name, a.last_name;
"""]

ia.sender(answer="sql_ex07", task="views", question="ex07", answer_type="pyvar")

## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos atores dos filmes mais rentáveis, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [ ]:
try:
    db('''
    SELECT actor_id, first_name, last_name
    FROM actor
        INNER JOIN film_actor USING (actor_id)
    WHERE film_id IN
        (SELECT film_id
        FROM (SELECT film_id, SUM(amount) as revenue
            FROM payment
            INNER JOIN rental USING (rental_id)
            GROUP BY film_id) as movie_revenue
        ORDER BY revenue DESC
        LIMIT 3);
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

In [ ]:
sql_ex08 = """
SELECT DISTINCT a.actor_id, a.first_name, a.last_name
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN (
    SELECT f.film_id, SUM(p.amount) AS gross_revenue
    FROM film f
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    JOIN payment p ON r.rental_id = p.rental_id
    GROUP BY f.film_id
    ORDER BY gross_revenue DESC
    LIMIT 3
) top_films ON fa.film_id = top_films.film_id
ORDER BY a.actor_id;

"""

db(sql_ex08)

In [ ]:
ia.sender(answer="sql_ex08", task="views", question="ex08", answer_type="pyvar")

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [ ]:
db("DROP TABLE IF EXISTS nomes_clientes")
db("""
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
""")

In [ ]:
db("DESCRIBE nomes_clientes")
db("SELECT * FROM nomes_clientes LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_staff")
db("""
CREATE TEMPORARY TABLE nomes_staff 
    SELECT first_name, last_name FROM staff
""")

In [ ]:
db("DESCRIBE nomes_staff")
db("SELECT * FROM nomes_staff LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_all")
db("""
CREATE TEMPORARY TABLE nomes_all 
    (SELECT * FROM nomes_staff)
    UNION 
    (SELECT * FROM nomes_clientes)
""")

In [ ]:
db("DESCRIBE nomes_all")
db("SELECT * FROM nomes_all LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_clientes")
db("DROP TABLE IF EXISTS nomes_staff")
db("DROP TABLE IF EXISTS nomes_all")

**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*. Ordene de forma ascendente por first_name e last_name.

In [ ]:
sql_ex09 = """
-- create a subqueries to get the first and last names of customers and staff
SELECT first_name, last_name
FROM customer
UNION
SELECT first_name, last_name
FROM staff
ORDER BY first_name, last_name;
"""

db(sql_ex09)

In [ ]:
ia.sender(answer="sql_ex09", task="views", question="ex09", answer_type="pyvar")

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme! Apresente o nome e a quantidade de atores. Ordene pelo id do filme. Dica: Pesquie a função `LEFT`

In [ ]:
sql_ex10 = """
-- create a list of movies that have more than two actors with the same inicial letter
SELECT f.title, COUNT(a.actor_id)
FROM film f, actor a, film_actor fa
WHERE f.film_id = fa.film_id 
    AND a.actor_id = fa.actor_id
    AND left(a.first_name, 1) = left(f.title, 1)
GROUP BY f.title
HAVING COUNT(a.actor_id) > 2;
"""
db(sql_ex10)

In [ ]:
ia.sender(answer="sql_ex10", task="views", question="ex10", answer_type="pyvar")

Semana do "DAN HARRIS": liste os clientes (nome e sobrenome) que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'. Ordene pelo nome e sobrenome do cliente.

Considerem que a pessoa pode ter assistido um filme de Dan Harris se:

- Devolveu em 2005-06-01 ou depois
- Alugou em 2005-06-01 ou depois
- Alugou antes de 2005-06-01 e ainda não devolveu

In [ ]:
# SELECT c.first_name, c.last_name
#     FROM customer c
#     WHERE c.first_name NOT IN
#         (SELECT DISTINCT c.first_name
#         FROM customer c, rental r, inventory i, film f
#         WHERE c.customer_id = r.customer_id
#             AND r.inventory_id = i.inventory_id
#             AND i.film_id = f.film_id
#             AND f.title = 'DAN HARRIS'
#             AND r.rental_date >= '2005-06-01')
#     AND c.first_name NOT IN
#         (SELECT DISTINCT c.first_name
#         FROM customer c, rental r, inventory i, film f
#         WHERE c.customer_id = r.customer_id
#             AND r.inventory_id = i.inventory_id
#             AND i.film_id = f.film_id
#             AND f.title = 'DAN HARRIS'
#             AND r.return_date >= '2005-06-01')
#     AND c.first_name NOT IN
#         (SELECT DISTINCT c.first_name
#         FROM customer c, rental r, inventory i, film f
#         WHERE c.customer_id = r.customer_id
#             AND r.inventory_id = i.inventory_id
#             AND i.film_id = f.film_id
#             AND f.title = 'DAN HARRIS'
#             AND r.rental_date >= '2005-06-01'
#             AND r.return_date = NULL)
#     ORDER BY c.first_name;

In [ ]:
sql_ex11 = """
SELECT c.first_name, c.last_name
FROM customer c
WHERE NOT EXISTS (
    SELECT 1
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film_actor fa ON i.film_id = fa.film_id
    JOIN actor a ON fa.actor_id = a.actor_id
    WHERE r.customer_id = c.customer_id
    AND a.first_name = 'DAN'
    AND a.last_name = 'HARRIS'
    AND (
        r.rental_date > '2005-06-01' -- rented after the specified date
        OR r.return_date > '2005-06-01' -- returned after the specified date
        OR r.return_date IS NULL -- hasn't returned yet
    )
)
ORDER BY c.first_name, c.last_name;
"""
db(sql_ex11)

In [ ]:
ia.sender(answer="sql_ex11", task="views", question="ex11", answer_type="pyvar")

- Faça uma consulta que retorne, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Ordene pelo nome e sobrenome do ator.

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [6]:
sql_ex12_old = """
SELECT 
    c.first_name,
    c.last_name,
    GROUP_CONCAT(CONCAT(category, ': ', CONCAT_WS(', ', filmes)) ORDER BY category SEPARATOR '; ') AS filmes_por_categoria
FROM
    customer c
JOIN
    (SELECT 
        c.customer_id,
            cat.name AS category,
            GROUP_CONCAT(DISTINCT f.title ORDER BY f.title SEPARATOR ', ') AS filmes
    FROM
        customer c
    JOIN rental r ON c.customer_id = r.customer_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category cat ON fc.category_id = cat.category_id
    GROUP BY c.customer_id, cat.name
    ORDER BY cat.name) 
    AS filmes_por_categoria ON c.customer_id = filmes_por_categoria.customer_id
GROUP BY c.customer_id
ORDER BY c.first_name, c.last_name;
"""

db(sql_ex12_old)

Executando query:
('AARON', 'SELBY', 'Action: DRIFTER COMMANDMENTS; Classics: CORE SUIT, PATIENT SISTER; Comedy: PERFECT GROOVE, SWEDEN SHINING; Drama: BEAUTY GREASE; Family: NETWORK PEAK, WILLOW TRACY; Foreign: ORANGE GRAPES, SHOCK CABIN, USUAL UNTOUCHABLES; Games: FEVER EMPIRE; Horror: ARACHNOPHOBIA ROLLERCOASTER, SLEEPING SUSPECTS, ZHIVAGO CORE; Music: DORADO NOTTING; Travel: DRUMLINE CYCLONE, EXPECATIONS NATURAL, FELLOWSHIP AUTUMN, LIAISONS SWEET, MUSCLE BRIGHT, TEEN APOLLO, VALENTINE VANISHING')
('ADAM', 'GOOCH', 'Action: BAREFOOT MANCHURIAN; Animation: TITANIC BOONDOCK; Children: HOLLYWOOD ANONYMOUS, POLISH BROOKLYN, STRANGELOVE DESIRE; Classics: JERK PAYCHECK, SPIKING ELEMENT; Comedy: SEARCHERS WAIT; Family: MUSIC BOONDOCK, SIEGE MADRE; Foreign: COMMAND DARLING, EVERYONE CRAFT, INFORMER DOUBLE; Games: FIRE WOLVES, NAME DETECTIVE; Horror: FREDDY STORM, KARATE MOON, SLEEPING SUSPECTS; Music: GROSSE WONDERFUL; New: PLUTO OLEANDER; Sports: ALADDIN CALENDAR, PEAK FOREVER')
('ADRIAN',

In [21]:
sql_ex12 = """
SELECT a.first_name, 
        a.last_name,
        GROUP_CONCAT(CONCAT(category, ': ', CONCAT_WS(', ', filmes)) ORDER BY category SEPARATOR '; ') AS filmes_por_categoria
    FROM actor a
    JOIN
        (SELECT 
            a.actor_id, c.name AS category,
            GROUP_CONCAT(DISTINCT f.title ORDER BY f.title SEPARATOR ', ') AS filmes
        FROM actor a
        JOIN film_actor fa ON a.actor_id = fa.actor_id
        JOIN film f ON fa.film_id = f.film_id
        JOIN film_category fc ON f.film_id = fc.film_id
        JOIN category c ON fc.category_id = c.category_id
        GROUP BY a.actor_id, c.name
        ORDER BY c.name
        ) AS filmes_por_categoria ON a.actor_id = filmes_por_categoria.actor_id
    GROUP BY a.actor_id
    ORDER BY a.first_name, a.last_name;
"""

db(sql_ex12)

Executando query:
('ADAM', 'GRANT', 'Action: MIDNIGHT WESTWARD; Children: IDOLS SNATCHERS, SPLENDOR PATTON, TWISTED PIRATES; Classics: TADPOLE PARK; Comedy: FIREBALL PHILADELPHIA, GROUNDHOG UNCUT, OPERATION OPERATION; Family: GLADIATOR WESTWARD; Foreign: BALLROOM MOCKINGBIRD, HAPPINESS UNITED; Games: GLORY TRACY, MARS ROMAN, WANDA CHAMBER; Sci-Fi: ANNIE IDENTITY; Sports: LOSER HUSTLER, SEABISCUIT PUNK; Travel: DISCIPLE MOTHER')
('ADAM', 'HOPPER', 'Action: CLUELESS BUCKET, MOCKINGBIRD HOLLYWOOD; Children: NOON PAPI; Classics: TOWERS HURRICANE; Comedy: HEAVEN FREEDOM, SADDLE ANTITRUST; Documentary: CLERKS ANGELS, PRINCESS GIANT; Drama: TORQUE BOUND; Family: BLOOD ARGONAUTS, GABLES METROPOLIS, GREASE YOUTH; Foreign: FICTION CHRISTMAS; Horror: TRAIN BUNCH; Music: CHAMBER ITALIAN, MASKED BUBBLE, WORDS HUNTER; New: LOVERBOY ATTACKS, SLEEPY JAPANESE; Sci-Fi: BLINDNESS GUN, OPEN AFRICAN, VACATION BOONDOCK')
('AL', 'GARLAND', 'Action: DRIFTER COMMANDMENTS, GLASS DYING, GRAIL FRANKENSTEIN, HANDI

In [19]:
ia.sender(answer="sql_ex12", task="views", question="ex12", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex12', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [ ]:
connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [22]:
ia.grades(by="TASK", task="views")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | views    |     10 |

In [23]:
ia.grades(task="views")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | views       | ex01        |      1 |     10 |
|  1 | views       | ex02        |      1 |     10 |
|  2 | views       | ex03        |      1 |     10 |
|  3 | views       | ex04        |      1 |     10 |
|  4 | views       | ex05        |      1 |     10 |
|  5 | views       | ex06        |      1 |     10 |
|  6 | views       | ex07        |      1 |     10 |
|  7 | views       | ex08        |      1 |     10 |
|  8 | views       | ex09        |      1 |     10 |
|  9 | views       | ex10        |      1 |     10 |
| 10 | views       | ex11        |      1 |     10 |
| 11 | views       | ex12        |      1 |     10 |

In [ ]:
ia.grades(by="task")